In [1]:
import numpy as np
import scipy.special
import cvxpy as cp

from scipy.optimize import minimize

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, IBMQ, execute
from qiskit.circuit import Parameter
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

from src import QAOA_utilities as QAOAut
from src import CellFreeQAOA as CFQAOA

# Loading your IBM Quantum account(s)
#provider = IBMQ.load_account()

In [2]:
M = 3 # number of cells
K = 4 # number of users
rho = 1 # max power
BW_max = 6 # max BW, M_hat in paper
rng = np.random.default_rng(270396)
#eta = np.random.rand(K) # power control fraction for each user
#alpha = np.random.rand(M,K)
#beta = np.random.rand(M,K)
eta = rng.random(K) # power control fraction for each user TO BE OTPIMIZED
alpha = rng.random((M,K))
beta = rng.random((M,K))

In [3]:
lambda_lagrange = rng.random(K)

----------------------------------------------------------------------------------------
Let's now consider the number of non-served users as a function of $t$.
------------------------------------------------------------------------------------------------------

In [4]:
t_vec = 10**(np.array([-20, -15, -10, -5, -1, 0])/10)

In [5]:
eta_samples = 3
alpha_samples = 3
beta_samples = 3

MCavg_nservedvec = np.zeros(len(t_vec))
MCavg_minSINRvec = np.zeros(len(t_vec))

for e in range(eta_samples):
    print('e = ',e)
    eta = rng.random(K) # power control fraction for each user TO BE OTPIMIZED
    for a in range(alpha_samples):
        print('a = ', a)
        alpha = rng.random((M,K))
        for b in range(beta_samples):
            print('b = ', b)
            beta = rng.random((M,K))

            nservedvec = np.zeros(len(t_vec))
            minSINRvec = np.zeros(len(t_vec))
            for k in range(len(t_vec)):
                t = t_vec[k]
                X_QAOA, avgSINR, avgserved, avgconst, counts = CFQAOA.optimize_links_fixedt(BW_max, rho, eta, alpha, beta, t, lambda_lagrange)
                print('t = ', t)
                print('avg served = ', avgserved)
                nservedvec[k] = avgserved
                print('avg SINR = ', avgSINR)
                minSINRvec[k] = min(avgSINR)
                print('----------------------------------------------------')
            MCavg_nservedvec += nservedvec
            MCavg_minSINRvec += minSINRvec

MCavg_minSINRvec = MCavg_minSINRvec/(eta_samples*alpha_samples*beta_samples)
MCavg_nservedvec = MCavg_nservedvec/(eta_samples*alpha_samples*beta_samples)
    

e =  0
a =  0
b =  0
----- Starting QAOA -----
COBYLA optimization result:  [1.57031786 1.26631903 1.70260016 1.64533996]
v best =  6
t =  0.01
avg served =  2.84375
avg SINR =  [0.03341465 0.06964682 0.17780103 0.34100221]
----------------------------------------------------
----- Starting QAOA -----
COBYLA optimization result:  [1.57742626 1.24717292 1.41977926 1.63534465]
v best =  7
t =  0.03162277660168379
avg served =  2.6015625
avg SINR =  [0.03297714 0.06904023 0.17585484 0.33697093]
----------------------------------------------------
----- Starting QAOA -----
COBYLA optimization result:  [1.65576659 1.15831322 0.86486405 1.91315664]
v best =  7
t =  0.1
avg served =  1.55078125
avg SINR =  [0.03306383 0.06927106 0.17012348 0.32710866]
----------------------------------------------------
----- Starting QAOA -----
COBYLA optimization result:  [1.33117612 0.44406268 0.50501918 0.45046867]
v best =  7
t =  0.31622776601683794
avg served =  1.126953125
avg SINR =  [0.02947252 0.06

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 

## Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(10*np.log10(t_vec), nservedvec)
plt.xlabel('log t (lower bound)')
plt.ylabel('log avg num served users')
plt.show()

In [ ]:
plt.plot(10*np.log10(t_vec), 10*np.log10(minSINRvec))
plt.xlabel('log t (lower bound)')
plt.ylabel('log avg min SINR')
plt.show()

1) Average connectivity over QAOA outputs
2) Optimize lambda classically (alternate optimization)
3) "Monte Carlo" over network params